# Section 1: Regime-Switching Strategy Introduction

This notebook implements, backtests, and analyzes a sophisticated regime-switching trading strategy. The core of this project is a custom-built, class-based, event-driven backtester designed to handle complex, path-dependent trading logic.

We will compare two variations of the strategy:
1.  **Baseline Strategy:** A simple regime model based on the VIX index.
2.  **Improved Strategy:** A more advanced regime model using a two-state Markov-switching model.

### Strategy Goal
The primary goal is to dynamically adapt the trading strategy based on the prevailing market volatility regime. By identifying whether the market is in a "Calm" or "Panic" state, we can apply the logic best suited for that environment.

-   **Assets:** `['NVDA', 'AAPL', 'MSFT', 'AMZN', 'META', 'AVGO', 'GOOGL', 'TSLA', 'GOOG', 'BRK-B']`
-   **Benchmark:** `SPY`

### Trading Logic

#### Regime 1: "Calm" (Low Volatility)
In periods of low volatility, markets tend to exhibit trending behavior. We will deploy a **Trend-Following** strategy.
-   **Logic:** Exponential Moving Average (EMA) Crossover.
-   **Go Long:** The short-term EMA (e.g., 20-period) crosses above the long-term EMA (e.g., 50-period).
-   **Exit Long:** The short-term EMA crosses below the long-term EMA.

#### Regime 2: "Panic" (High Volatility)
In periods of high volatility and market stress, prices often experience sharp drops followed by quick reversions. We will deploy a **Mean-Reversion** strategy to capitalize on these movements.
-   **Logic:** Bollinger Bands (e.g., 20-period, 2 standard deviations).
-   **Go Long ("Buy the Dip"):** The price touches or crosses below the Lower Bollinger Band.
-   **Exit Long ("Take Profit"):** The price reverts and touches the Middle Bollinger Band (the moving average).

### Critical Rule: Regime Switch
A core rule of the strategy is to manage risk during regime transitions. When the model detects a switch from one regime to another (e.g., Calm $\rightarrow$ Panic), **all open positions established under the *old* regime's logic are immediately closed.** This prevents holding a trend-following position in a market that has just entered a panic phase, or vice-versa.

# Section 2: Setup and Data Preparation

This section handles the initial setup, including importing necessary libraries, defining global parameters for the backtest, and downloading the required financial data from Yahoo Finance.

In [32]:
# 1. Imports
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from queue import Queue
from abc import ABC, abstractmethod
import datetime

# 2. Parameters
STOCK_LIST = ['NVDA', 'AAPL', 'MSFT', 'AMZN', 'META', 'AVGO', 'GOOGL', 'TSLA', 'GOOG', 'BRK-B']
BENCHMARK = 'SPY'
REGIME_INDICATOR = '^VIX'
FULL_START = '2010-01-01'
TRAIN_START = '2010-01-01'
TRAIN_END = '2017-12-31'
TEST_START = '2018-01-01'
TEST_END = '2019-12-31'
INITIAL_CAPITAL = 500000.0

# 3. Data Download Function
def download_data(tickers, start_date, end_date):
    """
    Downloads and cleans historical market data for a list of tickers.
    
    Args:
        tickers (list): List of ticker symbols.
        start_date (str): Start date for the data in 'YYYY-MM-DD' format.
        end_date (str): End date for the data in 'YYYY-MM-DD' format.
        
    Returns:
        dict: A dictionary of pandas DataFrames, where each key is a ticker
              and each value is its historical data.
    """
    data_container = {}
    all_tickers = tickers + [BENCHMARK, REGIME_INDICATOR]
    
    print(f"Downloading data for: {', '.join(all_tickers)}")
    
    raw_data = yf.download(all_tickers, start=start_date, end=end_date, group_by='ticker')
    raw_data.columns = raw_data.columns.get_level_values(0)
    
    for ticker in all_tickers:
        # yfinance returns a multi-level column index. We need to handle both
        # single ticker downloads and multi-ticker downloads.
        if len(all_tickers) > 1:
            df = raw_data[ticker].copy()
        else:
            df = raw_data.copy()

        # Clean data
        df.dropna(inplace=True)
        
        # Note on META/FB: yfinance automatically handles the ticker change from FB to META.
        # No special handling is required if the requested period overlaps with the change.
        
        data_container[ticker] = df
        
    print("Data download complete.")
    return data_container

# Execute the download
all_data = download_data(STOCK_LIST, FULL_START, TEST_END)

# Display a sample of the data for one stock
print("\nSample data for NVDA:")
print(all_data['NVDA'].head())

/var/folders/yl/gn9fldpj69169vhn466y4klh0000gn/T/ipykernel_14594/575873505.py:42: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_data = yf.download(all_tickers, start=start_date, end=end_date, group_by='ticker')
[*********************100%***********************]  12 of 12 completed

/var/folders/yl/gn9fldpj69169vhn466y4klh0000gn/T/ipykernel_14594/575873505.py:42: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_data = yf.download(all_tickers, start=start_date, end=end_date, group_by='ticker')
[*********************100%***********************]  12 of 12 completed

Data download complete.

Sample data for NVDA:
Ticker          NVDA      NVDA      NVDA      NVDA       NVDA
Date                                                         
2010-01-04  0.424289  0.426810  0.415120  0.423830  800204000
2010-01-05  0.422226  0.434604  0.422226  0.430019  728648000
2010-01-06  0.429790  0.433687  0.425664  0.432770  649168000
2010-01-07  0.430478  0.432312  0.421080  0.424289  547792000
2010-01-08  0.420850  0.428185  0.418329  0.425206  478168000


# Section 3: Class-Based Event-Driven Backtester Framework

This is the core of the project. We define a set of interacting classes that together form a robust, event-driven backtesting system. This architecture is highly extensible and allows for clear separation of concerns between data handling, strategy logic, portfolio management, and order execution.

-   **Event:** A simple object representing a state change in the system (e.g., new market data is available).
-   **EventQueue:** A central queue to hold all events, ensuring they are processed in order.
-   **DataHandler:** Responsible for providing market data for each time step of the backtest.
-   **Strategy:** Generates trading signals (`SignalEvent`) based on market data.
-   **Portfolio:** Manages positions, cash, and overall equity. It converts `SignalEvent`s into `OrderEvent`s.
-   **ExecutionHandler:** Simulates the execution of orders, converting `OrderEvent`s into `FillEvent`s.
-   **Backtest:** The main class that orchestrates the entire process, running the event loop and dispatching events to the appropriate handlers.

In [33]:
# 1. Event Classes
class Event:
    """
    Base class for all event objects.
    """
    pass

class MarketEvent(Event):
    """
    Handles the event of receiving new market data for a specific time.
    """
    def __init__(self, dt):
        self.type = 'MARKET'
        self.datetime = dt

class SignalEvent(Event):
    """
    Handles the event of sending a Signal from a Strategy object.
    This is received by a Portfolio object and acted upon.
    """
    def __init__(self, symbol, datetime, signal_type, strength=1.0):
        self.type = 'SIGNAL'
        self.symbol = symbol
        self.datetime = datetime
        self.signal_type = signal_type  # 'LONG', 'SHORT', 'EXIT'
        self.strength = strength

class OrderEvent(Event):
    """
    Handles the event of sending an Order to an execution system.
    The order contains a symbol, a type (market or limit), quantity and direction.
    """
    def __init__(self, symbol, order_type, quantity, direction):
        self.type = 'ORDER'
        self.symbol = symbol
        self.order_type = order_type  # 'MKT'
        self.quantity = quantity
        self.direction = direction  # 'BUY' or 'SELL'

    def print_order(self):
        print(f"Order: Symbol={self.symbol}, Type={self.order_type}, Quantity={self.quantity}, Direction={self.direction}")

class FillEvent(Event):
    """
    Encapsulates the notion of a filled order, as returned from a brokerage.
    Stores the quantity of an instrument actually filled and at what price.
    In addition, stores the commission of the trade from the brokerage.
    """
    def __init__(self, datetime, symbol, exchange, quantity, direction, fill_cost, commission=0.0):
        self.type = 'FILL'
        self.datetime = datetime
        self.symbol = symbol
        self.exchange = 'Simulated'
        self.quantity = quantity
        self.direction = direction
        self.fill_cost = fill_cost
        self.commission = commission

# 2. Event Queue
EventQueue = Queue()

# 3. DataHandler
class DataHandler(ABC):
    @abstractmethod
    def get_latest_bar(self, symbol):
        raise NotImplementedError("Should implement get_latest_bar()")

    @abstractmethod
    def get_latest_bars(self, symbol, N=1):
        raise NotImplementedError("Should implement get_latest_bars()")

    @abstractmethod
    def update_bars(self):
        raise NotImplementedError("Should implement update_bars()")

class HistoricYahooDataHandler(DataHandler):
    def __init__(self, events, tickers, start_date, end_date, data_source):
        self.events = events
        self.tickers = tickers
        self.start_date = pd.to_datetime(start_date)
        self.end_date = pd.to_datetime(end_date)
        self.data_source = data_source
        
        self.symbol_data = {ticker: self.data_source[ticker] for ticker in self.tickers + [BENCHMARK, REGIME_INDICATOR]}
        self.latest_symbol_data = {ticker: [] for ticker in self.tickers}
        
        # Use the SPY index as the trading calendar
        self.trading_days = self.data_source[BENCHMARK].loc[self.start_date:self.end_date].index
        self.current_day_index = 0

    def get_latest_bar(self, symbol):
        return self.latest_symbol_data[symbol][-1]

    def get_latest_bars(self, symbol, N=1):
        return self.latest_symbol_data[symbol][-N:]

    def update_bars(self):
        if self.current_day_index < len(self.trading_days):
            current_date = self.trading_days[self.current_day_index]
            
            for ticker in self.tickers:
                try:
                    bar = self.symbol_data[ticker].loc[current_date]
                    self.latest_symbol_data[ticker].append(bar)
                except KeyError:
                    # Handle cases where a stock might not have data for a specific day
                    pass
            
            self.events.put(MarketEvent(current_date))
            self.current_day_index += 1
        else:
            # Signal that there is no more data
            self.events.put(None)

# 4. Strategy
class Strategy(ABC):
    def __init__(self, data_handler, events):
        self.data_handler = data_handler
        self.events = events

    @abstractmethod
    def calculate_signals(self, event):
        raise NotImplementedError("Should implement calculate_signals()")

# 5. ExecutionHandler
class ExecutionHandler(ABC):
    @abstractmethod
    def execute_order(self, event):
        raise NotImplementedError("Should implement execute_order()")

class SimulatedExecutionHandler(ExecutionHandler):
    def __init__(self, events, data_handler):
        self.events = events
        self.data_handler = data_handler

    def execute_order(self, event):
        if event.type == 'ORDER':
            # Simulate fill based on the latest available close price
            bar = self.data_handler.get_latest_bar(event.symbol)
            fill_price = bar['Close']
            fill_cost = fill_price * event.quantity
            
            fill_event = FillEvent(
                bar.name, event.symbol, 'Simulated', event.quantity, 
                event.direction, fill_cost, commission=0.0
            )
            self.events.put(fill_event)

# 6. Portfolio
class Portfolio:
    def __init__(self, data_handler, events, initial_capital):
        self.data_handler = data_handler
        self.events = events
        self.initial_capital = initial_capital
        self.capital = initial_capital

        self.positions = {ticker: 0 for ticker in data_handler.tickers}
        self.holdings = {ticker: 0.0 for ticker in data_handler.tickers}
        
        self.all_positions = []
        self.all_holdings = []
        self.equity_curve = pd.DataFrame(columns=['total_value'])

    def update_timeindex(self, event):
        if event.type == 'MARKET':
            dt = event.datetime
            total_value = self.capital
            
            for ticker in self.positions:
                if self.positions[ticker] != 0:
                    bar = self.data_handler.get_latest_bar(ticker)
                    market_value = self.positions[ticker] * bar['Close']
                    self.holdings[ticker] = market_value
                    total_value += market_value
            
            self.equity_curve.loc[dt, 'total_value'] = total_value

    def update_fill(self, event):
        if event.type == 'FILL':
            if event.direction == 'BUY':
                self.positions[event.symbol] += event.quantity
                self.capital -= event.fill_cost
            elif event.direction == 'SELL':
                self.positions[event.symbol] -= event.quantity
                self.capital += event.fill_cost

    def generate_naive_order(self, signal):
        if signal.type == 'SIGNAL':
            symbol = signal.symbol
            direction = signal.signal_type
            
            # Equal allocation: 1/N of total portfolio value per trade
            target_investment = self.equity_curve.iloc[-1]['total_value'] / len(self.data_handler.tickers)
            
            current_position = self.positions[symbol]
            bar = self.data_handler.get_latest_bar(symbol)
            price = bar['Close']
            
            if direction == 'LONG' and current_position == 0:
                quantity = int(target_investment / price)
                if quantity > 0:
                    order = OrderEvent(symbol, 'MKT', quantity, 'BUY')
                    self.events.put(order)
            elif direction == 'EXIT' and current_position > 0:
                order = OrderEvent(symbol, 'MKT', current_position, 'SELL')
                self.events.put(order)

    def get_equity_curve(self):
        return self.equity_curve

# 7. Backtest
class Backtest:
    def __init__(self, data_handler, strategy, portfolio, execution_handler, events):
        self.data_handler = data_handler
        self.strategy = strategy
        self.portfolio = portfolio
        self.execution_handler = execution_handler
        self.events = events

    def run_backtest(self):
        print("Starting backtest...")
        while True:
            self.data_handler.update_bars()
            
            while True:
                try:
                    event = self.events.get(block=False)
                except Queue.Empty:
                    break
                else:
                    if event is not None:
                        if event.type == 'MARKET':
                            self.strategy.calculate_signals(event)
                            self.portfolio.update_timeindex(event)
                        elif event.type == 'SIGNAL':
                            self.portfolio.generate_naive_order(event)
                        elif event.type == 'ORDER':
                            self.execution_handler.execute_order(event)
                        elif event.type == 'FILL':
                            self.portfolio.update_fill(event)
            
            if self.data_handler.current_day_index >= len(self.data_handler.trading_days):
                break
        print("Backtest finished.")

# Section 4: Strategy Implementation

Here we define the `RegimeSwitchingStrategy`, which is the brain of our trading system. This class inherits from the abstract `Strategy` class and implements the `calculate_signals` method.

Its key responsibilities are:
1.  **Regime Calculation (in `__init__`):** Before the backtest starts, it pre-calculates the regime state (Calm or Panic) for every day in our dataset using both the VIX and Markov models. This is a crucial step for efficiency.
2.  **Signal Generation (in `calculate_signals`):** On every market event (i.e., for each new day), it checks the current regime, looks for trading signals based on the corresponding logic (EMA crossover or Bollinger Bands), and checks for regime switches that would trigger closing all positions.

In [34]:
class RegimeSwitchingStrategy(Strategy):
    def __init__(self, data_handler, events, strategy_params):
        super().__init__(data_handler, events)
        self.params = strategy_params
        self.regime_mode = self.params['regime_mode']
        
        self._calculate_regimes()

    def _calculate_regimes(self):
        """
        Pre-calculates the VIX and Markov regimes for the entire dataset.
        """
        print("Calculating regimes...")
        # VIX Regime
        vix_data = self.data_handler.symbol_data[REGIME_INDICATOR]
        self.vix_regime = pd.Series(np.where(vix_data['Close'] > self.params['vix_threshold'], 2, 1), index=vix_data.index)
        
        # Markov Regime
        # 1. Create equal-weighted portfolio index
        portfolio_df = pd.DataFrame(index=self.data_handler.trading_days)
        for ticker in self.data_handler.tickers:
            portfolio_df[ticker] = self.data_handler.symbol_data[ticker]['Close']
        
        portfolio_returns = portfolio_df.pct_change().mean(axis=1).dropna()
        
        # 2. Fit Markov model on training data
        train_returns = portfolio_returns.loc[TRAIN_START:TRAIN_END]
        model = sm.tsa.MarkovRegression(train_returns, k_regimes=2, trend='c', switching_variance=True)
        
        print("Fitting Markov Model...")
        res = model.fit(iter=1000)
        
        # Identify high-volatility state
        high_vol_regime = np.argmax(res.params[-2:]) # Last two params are sigmas
        
        # 3. Predict on full dataset
        full_returns = portfolio_returns.loc[FULL_START:TEST_END]
        self.markov_regime_probs = model.predict(res.params, full_returns)
        self.markov_regime = pd.Series(
            np.where(self.markov_regime_probs[high_vol_regime] > self.params['markov_threshold'], 2, 1),
            index=full_returns.index
        )
        
        # Select the regime series to be used in the backtest
        if self.regime_mode == 'VIX':
            self.regime_series = self.vix_regime
        elif self.regime_mode == 'MARKOV':
            self.regime_series = self.markov_regime
            
        print("Regime calculation complete.")

    def calculate_signals(self, event):
        if event.type == 'MARKET':
            dt = event.datetime
            
            # Check for regime switch
            try:
                current_regime = self.regime_series.loc[dt]
                yesterday = self.regime_series.index[self.regime_series.index.get_loc(dt) - 1]
                yesterday_regime = self.regime_series.loc[yesterday]

                if current_regime != yesterday_regime:
                    for symbol in self.data_handler.tickers:
                        if self.data_handler.portfolio.positions[symbol] != 0:
                            signal = SignalEvent(symbol, dt, 'EXIT')
                            self.events.put(signal)
            except (KeyError, IndexError):
                pass # Not enough data yet

            # Apply regime-specific rules
            for symbol in self.data_handler.tickers:
                bars = self.data_handler.get_latest_bars(symbol, N=self.params['ema_long'] + 1)
                if len(bars) < self.params['ema_long']:
                    continue

                close_prices = pd.Series([b['Close'] for b in bars])

                if current_regime == 1: # Calm -> Trend-Following
                    ema_short = close_prices.ewm(span=self.params['ema_short']).mean().iloc[-1]
                    ema_long = close_prices.ewm(span=self.params['ema_long']).mean().iloc[-1]
                    
                    # Check for crossover
                    if ema_short > ema_long and self.data_handler.portfolio.positions[symbol] == 0:
                        signal = SignalEvent(symbol, dt, 'LONG')
                        self.events.put(signal)
                    elif ema_short < ema_long and self.data_handler.portfolio.positions[symbol] > 0:
                        signal = SignalEvent(symbol, dt, 'EXIT')
                        self.events.put(signal)

                elif current_regime == 2: # Panic -> Mean-Reversion
                    bb_period = self.params['bband_period']
                    bb_std = self.params['bband_std']
                    
                    if len(close_prices) < bb_period:
                        continue
                        
                    middle_band = close_prices.rolling(window=bb_period).mean().iloc[-1]
                    std_dev = close_prices.rolling(window=bb_period).std().iloc[-1]
                    lower_band = middle_band - (bb_std * std_dev)
                    
                    current_price = close_prices.iloc[-1]
                    
                    if current_price <= lower_band and self.data_handler.portfolio.positions[symbol] == 0:
                        signal = SignalEvent(symbol, dt, 'LONG')
                        self.events.put(signal)
                    elif current_price >= middle_band and self.data_handler.portfolio.positions[symbol] > 0:
                        signal = SignalEvent(symbol, dt, 'EXIT')
                        self.events.put(signal)

# Section 5: Analysis and Helper Functions

To analyze the performance of our strategies and to facilitate the optimization process, we create a set of helper functions.

-   **`run_backtest_helper`:** A crucial wrapper that takes a configuration dictionary, sets up all the necessary components of the backtester (DataHandler, Strategy, Portfolio, etc.), runs the backtest, and returns the resulting portfolio object for analysis.
-   **`calculate_performance_metrics`:** This function computes a variety of standard financial metrics (CAGR, Sharpe Ratio, Max Drawdown, etc.) from a portfolio's equity curve. This allows for quantitative comparison between different strategies and the benchmark.
-   **`plot_equity_curves`:** A visualization utility to plot the equity curves of multiple strategies on a single chart, providing an immediate visual comparison of their performance over time.

In [35]:
# 1. Backtest Helper Function
def run_backtest_helper(tickers, start_date, end_date, initial_capital, strategy_params):
    """
    A wrapper function to set up and run a backtest for a given configuration.
    """
    events = Queue()
    
    data_handler = HistoricYahooDataHandler(events, tickers, start_date, end_date, all_data)
    portfolio = Portfolio(data_handler, events, initial_capital)
    strategy = RegimeSwitchingStrategy(data_handler, events, strategy_params)
    execution_handler = SimulatedExecutionHandler(events, data_handler)
    
    # Wire the portfolio to the data_handler so the strategy can access it
    data_handler.portfolio = portfolio

    backtest = Backtest(data_handler, strategy, portfolio, execution_handler, events)
    backtest.run_backtest()
    
    return portfolio

# 2. Performance Metrics Calculation
def calculate_performance_metrics(equity_curve, benchmark_returns):
    """
    Calculates and returns a DataFrame of performance metrics.
    """
    metrics = {}
    
    returns = equity_curve['total_value'].pct_change().dropna()
    
    # Total Return
    metrics['Total Return'] = (equity_curve['total_value'].iloc[-1] / equity_curve['total_value'].iloc[0]) - 1
    
    # CAGR
    days = (equity_curve.index[-1] - equity_curve.index[0]).days
    metrics['CAGR'] = (1 + metrics['Total Return']) ** (365.0 / days) - 1
    
    # Volatility (Annualized)
    metrics['Volatility'] = returns.std() * np.sqrt(252)
    
    # Sharpe Ratio (Annualized)
    # Assuming risk-free rate is 0
    metrics['Sharpe Ratio'] = metrics['CAGR'] / metrics['Volatility']
    
    # Sortino Ratio (Annualized)
    downside_returns = returns[returns < 0]
    downside_std = downside_returns.std() * np.sqrt(252)
    metrics['Sortino Ratio'] = metrics['CAGR'] / downside_std
    
    # Max Drawdown
    roll_max = equity_curve['total_value'].cummax()
    drawdown = (equity_curve['total_value'] - roll_max) / roll_max
    metrics['Max Drawdown'] = drawdown.min()
    
    # Alpha and Beta
    merged = pd.DataFrame({'strategy': returns, 'benchmark': benchmark_returns}).dropna()
    cov = merged.cov().iloc[0, 1]
    beta = cov / merged['benchmark'].var()
    alpha = metrics['CAGR'] - beta * (benchmark_returns.mean() * 252)
    
    metrics['Alpha'] = alpha
    metrics['Beta'] = beta
    
    return pd.DataFrame(metrics, index=[0])

# 3. Plotting Function
def plot_equity_curves(results_dict, title):
    """
    Plots multiple equity curves on a single chart.
    """
    plt.figure(figsize=(12, 8))
    for name, curve in results_dict.items():
        normalized_curve = curve['total_value'] / curve['total_value'].iloc[0]
        plt.plot(normalized_curve, label=name)
        
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.grid(True)
    plt.show()

# Section 6: Parameter Optimization (Grid Search)

To enhance the performance of our **Improved (Markov) Strategy**, we will perform a simple grid search to find the optimal parameters for our trading logic. This optimization is performed *only* on the **Training Period** (`2010-01-01` to `2017-12-31`) to avoid lookahead bias. The goal is to find the combination of parameters that results in the highest Sharpe Ratio on this training data.

The parameters we will optimize are:
-   The short and long periods for the EMA Crossover (Trend-Following) strategy.
-   The period and standard deviation for the Bollinger Bands (Mean-Reversion) strategy.

In [36]:
# 1. Grid Definition
ema_params = [(10, 30), (20, 50), (30, 60)]
bband_params = [(20, 2.0), (30, 2.0), (20, 2.5)]

best_sharpe = -np.inf
best_params = {}

# 2. Search Loop
print("Starting grid search for best parameters on training data...")
for ema_short, ema_long in ema_params:
    for bband_period, bband_std in bband_params:
        
        current_params = {
            'regime_mode': 'MARKOV',
            'ema_short': ema_short,
            'ema_long': ema_long,
            'bband_period': bband_period,
            'bband_std': bband_std,
            'vix_threshold': 20,
            'markov_threshold': 0.5
        }
        
        print(f"Testing params: EMA({ema_short},{ema_long}), BBand({bband_period},{bband_std})")
        
        # Run backtest on training data
        portfolio = run_backtest_helper(
            tickers=STOCK_LIST,
            start_date=TRAIN_START,
            end_date=TRAIN_END,
            initial_capital=INITIAL_CAPITAL,
            strategy_params=current_params
        )
        
        equity_curve = portfolio.get_equity_curve()
        
        # Calculate Sharpe Ratio
        returns = equity_curve['total_value'].pct_change().dropna()
        if not returns.empty:
            volatility = returns.std() * np.sqrt(252)
            if volatility > 0:
                cagr = (1 + ((equity_curve['total_value'].iloc[-1] / equity_curve['total_value'].iloc[0]) - 1)) ** (365.0 / (equity_curve.index[-1] - equity_curve.index[0]).days) - 1
                sharpe = cagr / volatility
                
                if sharpe > best_sharpe:
                    best_sharpe = sharpe
                    best_params = current_params

print("\n--- Grid Search Complete ---")
print(f"Best Sharpe Ratio on Training Data: {best_sharpe:.4f}")
print("Best Parameters Found:")
print(best_params)

# Store the best params for the final run
best_params_from_training = best_params

Starting grid search for best parameters on training data...
Testing params: EMA(10,30), BBand(20,2.0)
Calculating regimes...


Starting grid search for best parameters on training data...
Testing params: EMA(10,30), BBand(20,2.0)
Calculating regimes...


KeyError: 'Close'

# Section 7: Final Backtest & Performance Comparison

Now we conduct the final out-of-sample test on the **Test Period** (`2018-01-01` to `2019-12-31`). This is the true test of our strategy's viability, as it uses data that was not seen during the model fitting or parameter optimization stages.

We will run three backtests:
1.  **Benchmark:** The `SPY` ETF, representing the broader market.
2.  **Baseline Strategy:** The VIX-based regime-switching model using default parameters.
3.  **Improved Strategy:** The Markov-based regime-switching model using the `best_params` discovered during our grid search.

Finally, we will collate the results, generate a table of performance metrics, and plot the equity curves for a clear visual comparison.

In [ ]:
# 1. Run Benchmark
spy_data = all_data[BENCHMARK].loc[TEST_START:TEST_END]
spy_returns = spy_data['Close'].pct_change().dropna()
spy_equity_curve = (1 + spy_returns).cumprod() * INITIAL_CAPITAL
spy_equity_curve = pd.DataFrame({'total_value': spy_equity_curve})


# 2. Run Baseline (VIX) Strategy
print("\nRunning Baseline (VIX) Strategy on Test Data...")
vix_params = {
    'regime_mode': 'VIX',
    'ema_short': 20,
    'ema_long': 50,
    'bband_period': 20,
    'bband_std': 2.0,
    'vix_threshold': 20,
    'markov_threshold': 0.5
}
vix_portfolio = run_backtest_helper(
    tickers=STOCK_LIST,
    start_date=TEST_START,
    end_date=TEST_END,
    initial_capital=INITIAL_CAPITAL,
    strategy_params=vix_params
)
vix_equity_curve = vix_portfolio.get_equity_curve()

# 3. Run Improved (Markov) Strategy
print("\nRunning Improved (Markov) Strategy on Test Data...")
markov_portfolio = run_backtest_helper(
    tickers=STOCK_LIST,
    start_date=TEST_START,
    end_date=TEST_END,
    initial_capital=INITIAL_CAPITAL,
    strategy_params=best_params_from_training
)
markov_equity_curve = markov_portfolio.get_equity_curve()

# 4. Collate Results
vix_metrics = calculate_performance_metrics(vix_equity_curve, spy_returns)
vix_metrics.index = ['VIX Strategy']

markov_metrics = calculate_performance_metrics(markov_equity_curve, spy_returns)
markov_metrics.index = ['Markov Strategy']

spy_metrics = calculate_performance_metrics(spy_equity_curve, spy_returns)
spy_metrics.index = ['SPY Benchmark']

all_metrics = pd.concat([vix_metrics, markov_metrics, spy_metrics])

# 5. Visualize
print("\n--- Performance Comparison (Test Period) ---")
print(all_metrics)

plot_equity_curves(
    {'VIX Strategy': vix_equity_curve, 'Markov Strategy': markov_equity_curve, 'SPY Benchmark': spy_equity_curve},
    'Strategy Performance Comparison (Out-of-Sample)'
)

# Section 8: Strategy Critique

This section provides a qualitative critique of the **Improved (Markov) Strategy**, highlighting its strengths, weaknesses, and potential avenues for future improvement.

### Benefits

*   **Data-Driven and Adaptive:** Unlike the static VIX threshold, the Markov model learns the volatility characteristics directly from the portfolio's own returns. This makes it more tailored to the specific assets being traded.
*   **Less Prone to Whipsaw:** The VIX can be very spiky, potentially causing frequent and unprofitable regime switches around the threshold. The Markov model's smoothed probabilities often lead to more stable and meaningful regime classifications.
*   **Forward-Looking Probabilities:** The model provides probabilities of being in a certain state, which can be used for more nuanced decision-making (e.g., scaling positions based on the certainty of the regime) rather than a binary switch.

### Limitations

*   **Model Risk:** The model's effectiveness is entirely dependent on how well it fits the data. If the underlying market dynamics do not conform to a two-state Markov-switching process, the model's predictions will be unreliable.
*   **Overfitting Risk:** While we used a train/test split, the grid search for parameters still carries a risk of overfitting to the training data. The chosen parameters might not be optimal for future market conditions.
*   **Assumption of Two States:** We assumed the market can be neatly divided into two regimes (Calm and Panic). In reality, there may be more states (e.g., Normal, High-Vol, Crash), and a two-state model might oversimplify the market's behavior.

### Opportunities for Improvement

*   **More Sophisticated Models:** Incorporate more variables into the Markov model, such as trading volume or the VIX itself, to create a richer, multivariate regime definition. A three-state model could also be explored.
*   **Dynamic Position Sizing:** Instead of equal allocation, position size could be adjusted based on the strength of the signal or the probability of being in a particular regime from the Markov model.
*   **Advanced Risk Management:** Implement more dynamic stop-loss mechanisms, such as ATR (Average True Range)-based stops, that adjust to the volatility of each specific regime.

### Challenges

*   **Regime Instability (Concept Drift):** The statistical properties of market regimes can change over time. A model trained on data from 2010-2017 might become less effective in the 2020s. The model would likely need to be periodically refit on new data to remain relevant.
*   **Parameter Sensitivity:** The performance of the strategy can be highly sensitive to the chosen parameters (EMAs, Bollinger Bands, thresholds). The optimal parameters from the past are not guaranteed to be optimal in the future.

# Section 9: Final Execution Cell

This final cell is designed for a clean, final run of the **Improved (Markov) Strategy** on the specified test period. It uses the optimal parameters discovered during the grid search phase.

In [ ]:
# --- FINAL EXECUTION CELL ---

# Parameters for the confidential run
start_date = "2018-01-01"  # Test Start
end_date = "2019-12-31"    # Test End
stock_list = ['NVDA', 'AAPL', 'MSFT', 'AMZN', 'META', 'AVGO', 'GOOGL', 'TSLA', 'GOOG', 'BRK-B']
initial_capital = 500000.0
transaction_costs = 0.0
leverage = 0.0

# Use the best parameters found during the grid search
# If the grid search did not run or find params, use a default set.
if 'best_params_from_training' not in locals() or not best_params_from_training:
    print("Warning: Best parameters not found from grid search. Using defaults.")
    best_params_from_training = {
        'regime_mode': 'MARKOV', 'ema_short': 20, 'ema_long': 50,
        'bband_period': 20, 'bband_std': 2.0, 'vix_threshold': 20, 'markov_threshold': 0.5
    }

print("Running final backtest on test data with optimal parameters...")
final_portfolio = run_backtest_helper(
    tickers=stock_list,
    start_date=start_date,
    end_date=end_date,
    initial_capital=initial_capital,
    strategy_params=best_params_from_training
)

print("\nCalculating final performance metrics...")
# Get benchmark returns for the same period
spy_returns_final = all_data[BENCHMARK]['Close'].loc[start_date:end_date].pct_change().dropna()
final_equity_curve = final_portfolio.get_equity_curve()

if not final_equity_curve.empty:
    final_metrics = calculate_performance_metrics(final_equity_curve, spy_returns_final)

    print("\n--- Final Strategy Performance (Test Period) ---")
    print(final_metrics)
else:
    print("Final equity curve is empty. No trades were made.")